# 03 Embeddings | 01 Basic Embeddings

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.16"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;

//configuration file is created during environment creation
static string _configurationFile = @"../Configuration/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_EMBEDDING_DEPLOYMENTNAME not found";

string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "assets");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);
Console.WriteLine($"OpenAI Client created...");


Installed Packages Azure.AI.OpenAI, 1.0.0-beta.16 DotNetEnv, 2.5.0

OpenAI Client created...


## Step 2: Create Embeddings

The following sample creates a vector representation (embedding) from input text which should be stored in a data base and a query to perform a semantic search.
The sample uses the Azure OpenAI SDK.

To calculate the embeddings/vector you have to call the method `GetEmbeddingsAsync()` on the instance of `OpenAIClient`. The method provides multiple return values where the embedding can be found in `modelResponse.Value.Data[0]`.

In the example embeddings two embeddings/vectors are calculated using the deployed OpenAI model:
- The file `Path.Combine(assetsFolder, "Embedding", "TheUntouchableAdolescents_Part01.txt");` is part of a free e-book downloaded from the [Project Gutenberg](www.gutenberg.com)
- A hard coded string `"Did Shreve gave a bitter laugh, thin and short?"` will be also transformed into an embedding/vector representation. The phrase is used at the end of the text file.

Both embeddings will be used in further samples to perform a vector based query.

In [2]:

string documentationPage = Path.Combine(assetsFolder,"Embedding", "TheUntouchableAdolescents_Part01.txt");

string textToBeVecorized = File.ReadAllText(documentationPage);

// Vectorize input text from file
EmbeddingsOptions embeddingsOptions = new EmbeddingsOptions(embeddingDeploymentName, new List<string> { textToBeVecorized });
var modelResponse = await openAIClient.GetEmbeddingsAsync( embeddingsOptions);
float[] vectorDocumentation = modelResponse.Value.Data[0].Embedding.ToArray();

Console.WriteLine($"Vector from {documentationPage} created... Vector Length: {vectorDocumentation.Length}");

// Vectorize question
string question = "Did Shreve gave a bitter laugh, thin and short?";
embeddingsOptions = new EmbeddingsOptions(embeddingDeploymentName, new List<string> { question });
modelResponse = await openAIClient.GetEmbeddingsAsync(embeddingsOptions);
float[] vectorQuery = modelResponse.Value.Data[0].Embedding.ToArray();

Console.WriteLine($"Vector from '{question}' created... Vector Length: {vectorQuery.Length}");


Vector from c:\Sourcen\GitHubProjects\OpenAI.Workshop\03_Embedding\..\assets\Embedding\TheUntouchableAdolescents_Part01.txt created...
Vector from 'Did Shreve gave a bitter laugh, thin and short?' created... Vector Length: 1536


## Step 3: Vector storage

Both created vectors are stored in a txt file for further analyze and/or usage.

In [5]:
//Store both Vectors
string dataVectorFileName = "../Assets/Embedding/DataVector.txt";
string queryVectorFileName = "../Assets/Embedding/QueryVector.txt";

await File.WriteAllLinesAsync(dataVectorFileName, vectorDocumentation.Select(x => x.ToString()));
await File.WriteAllLinesAsync(queryVectorFileName, vectorQuery.Select(x => x.ToString()));

Console.WriteLine($"Vectors stored in {dataVectorFileName} and {queryVectorFileName}...");

Vectors stored in ../Assets/Embedding/DataVector.txt and ../Assets/Embedding/QueryVector.txt...
